## Using the NLP API in the notebook

In [17]:
text1 = """{'HISTORY': 'HISTORY: Low back pain for a few months with pain radiating to left leg.\n', 'TECHNIQUE': 'TECHNIQUE: Sagittal T1 and T2 and axial T2-weighted sequences were obtained through the\nlumbar spine without IV contrast.\n', 
'FINDINGS': 'FINDINGS: There is no focal marrow-replacing process identified. There is no fracture\nidentified. The conus medullaris appears in the appropriate position. The distal thoracic cord is\nunremarkable. The sacrum is homogeneous. The STIR sequences demonstrate no bone marrow\nedema or fracture.\nAt T12-L1, there is no canal stenosis identified. The disc space height is maintained. The neural\nforamina are patent.\nAt L1-2, there is no canal stenosis identified. The disc space height is maintained. The neural\nforamina are patent.\nAt L2-3, there is no canal stenosis identified. The disc space height is maintained. The neural\nforamina are patent.\nAt L3-4, there is no canal stenosis identified. The disc space height is maintained. The neural\nforamina are patent.\nAt L4-5, there is a disc herniation of protrusion type toward the left which does encroach upon\nthe proximal left L5 nerve root. The neural foramina demonstrate mild narrowing bilaterally.\nThere facet arthropathy identified, mild to moderate in severity.\nContinued on the following page.\nAt L5-S1, there is no canal stenosis. The disc space height is maintained. The neural foramina\nare patent.\n', 
'IMPRESSION': 'IMPRESSION:\n1. L4-5 disc bulge and disc herniation of protrusion type with mild canal stenosis and\nencroachment on the proximal left L5 nerve root. The neural foramina demonstrate mild\nnarrowing bilaterally.\n2. Loss of the lumbar lordosis. There is no fracture identified.\n'}"""

text1 = text1.replace("\n", "")

text2 = """{'HISTORY': 'HISTORY: Chronic neck pain.\n', 
'TECHNIQUE': 'TECHNIQUE: Sagittal T1, T2 gradient-echo, axial gradient-echo sequences were obtained through the\ncervical spine. Additional MR myelographic sequence was performed.\n', 
'COMPARISON': 'COMPARISON: No prior films are available for comparison.\n', 
'FINDINGS': 'FINDINGS: There is loss of cervical lordosis is identified. Posterior spinous processes are intact. There\nis loss of disc space height at C5-6 and C6-7 with osteophyte extending anteriorly. No syrinx is identified\nin the cord.\nThe ring of C1 is intact. Atlantoaxial joint has hypertrophic changes. Cervical ligaments at the base of\nthe skull appear intact.\nIn the upper thoracic spine, there is no canal stenosis identified.\nSTIR sequences demonstrate no significant bone marrow edema.\nThe facets appear unremarkable. Alar ligaments demonstrate no hyperintense signal.\nAt C2-3, there is disc bulge diffusely and facet hypertrophy, moderate to marked in severity. There is\nmild left foraminal narrowing identified.\nAt C3-4, there is focal disc herniation protrusion type identified with mild canal stenosis. The neural\nforamina demonstrate narrowing towards the left, moderate-to-marked in severity with uncovertebral joint\nhypertrophy and facet arthropathy identified.\nAt C4-5, there is disc bulge and disc herniation to the right with moderate canal stenosis and cord\nindentation with foraminal narrowing identified bilaterally, worse towards the right marked in severity.\nThere is facet arthropathy, worse towards the right side.\nContinued on the following page.\nAt C5-6, there is disc bulge and disc herniation identified with moderate to severe canal stenosis and cord\nindentation with foraminal narrowing bilaterally, marked in severity worse towards the right.\nAt C6-7, there is central disc herniation protrusion type with moderate to severe canal stenosis and cord\nindentation with foraminal narrowing identified bilaterally, worse towards the left moderate in severity.\nAt C7-T1, disc herniation centrally is identified with moderate canal stenosis identified. There is\nanterolisthesis of 3 mm identified. There is foraminal narrowing identified bilaterally, moderate-to-\nmarked in severity worse towards the left side. Uncovertebral joint hypertrophy and facet\nosteoarthropathy is identified.\nThere is scattered lymphadenopathy identified throughout the neck and clinical correlation is\nrecommended to assess this adenopathy further. Lymphadenopathy is worst towards the left\nsupraclavicular region.\n', 
'IMPRESSION': 'IMPRESSION:\n1. C5-6 disc bulge and disc herniation to the right with moderate to severe canal stenosis and cord\nindentation with foraminal narrowing bilaterally, worse towards the marked in severity.\n2. C6-7 disc bulge is identified with disc herniation and moderate to severe canal stenosis with cord\nindentation and foraminal narrowing identified bilaterally, mild towards the right and moderate\ntowards the left.\n3. C4-5 disc bulge and disc herniation to the right with moderate canal stenosis and cord indentation\nwith foraminal narrowing identified, worse towards the right marked in severity.\n4. C3-4 focal disc herniation protrusion type without canal stenosis.\n5. There is lymphadenopathy in the neck identified with dominant area of apparent left-sided\nsupraclavicular adenopathy identified measuring approximately 2.7 cm. Correlation with CT\nneck with contrast is recommended to assess this adenopathy further as underlying lymphoma\nwould be of utmost concern given the size of the lymphadenopathy that is present.\n'}
"""
text2 = text2.replace("\n", "")
print(text1)
print("----------------------\n \n")

print(text2)
print("----------------------\n \n")

{'HISTORY': 'HISTORY: Low back pain for a few months with pain radiating to left leg.', 'TECHNIQUE': 'TECHNIQUE: Sagittal T1 and T2 and axial T2-weighted sequences were obtained through thelumbar spine without IV contrast.', 'FINDINGS': 'FINDINGS: There is no focal marrow-replacing process identified. There is no fractureidentified. The conus medullaris appears in the appropriate position. The distal thoracic cord isunremarkable. The sacrum is homogeneous. The STIR sequences demonstrate no bone marrowedema or fracture.At T12-L1, there is no canal stenosis identified. The disc space height is maintained. The neuralforamina are patent.At L1-2, there is no canal stenosis identified. The disc space height is maintained. The neuralforamina are patent.At L2-3, there is no canal stenosis identified. The disc space height is maintained. The neuralforamina are patent.At L3-4, there is no canal stenosis identified. The disc space height is maintained. The neuralforamina are patent.At L4-5, there

In [29]:
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from google.auth import compute_engine
import json

PROJECT_ID="aug2022-361002"
URL="https://healthcare.googleapis.com/v1/projects/"+PROJECT_ID+"/locations/us-central1/services/nlp:analyzeEntities"
NLP_SERVICE='projects/'+PROJECT_ID+'/locations/us-central1/services/nlp'

# Define the required scopes
scopes = [
    "https://www.googleapis.com/auth/cloud-platform"
]

# Authenticate a credential with the service account
credentials = compute_engine.Credentials()

# Use the credentials object to authenticate a Requests session.
authed_session = AuthorizedSession(credentials)

payload = {
    'nlpService': NLP_SERVICE,
    'documentContent': text2 #'Insulin regimen human 5 units IV administered.'
}
# Use the credentials object to authenticate a Requests session.
url = URL
print(url)
print(payload)
headers = {"Accept": "application/json"}
response = authed_session.post(
    url,
    json = payload,
    headers=headers
)

data = response.json()
# Writing to sample.json
with open("text2.json", "w") as outfile:
    outfile.write(json.dumps(data, indent=4))

https://healthcare.googleapis.com/v1/projects/aug2022-361002/locations/us-central1/services/nlp:analyzeEntities
{'nlpService': 'projects/aug2022-361002/locations/us-central1/services/nlp', 'documentContent': "{'HISTORY': 'HISTORY: Chronic neck pain.', 'TECHNIQUE': 'TECHNIQUE: Sagittal T1, T2 gradient-echo, axial gradient-echo sequences were obtained through thecervical spine. Additional MR myelographic sequence was performed.', 'COMPARISON': 'COMPARISON: No prior films are available for comparison.', 'FINDINGS': 'FINDINGS: There is loss of cervical lordosis is identified. Posterior spinous processes are intact. Thereis loss of disc space height at C5-6 and C6-7 with osteophyte extending anteriorly. No syrinx is identifiedin the cord.The ring of C1 is intact. Atlantoaxial joint has hypertrophic changes. Cervical ligaments at the base ofthe skull appear intact.In the upper thoracic spine, there is no canal stenosis identified.STIR sequences demonstrate no significant bone marrow edema.Th

## Printing the raw data. To better see the results, take this and use a json viewer such as 
http://jsonviewer.stack.hu/

In [ ]:
print(json.dumps(data, indent = 4))

In [23]:
import pandas as pd
column_names = ["Entity#", "Entity type", "Content", "Entity confidence","Temporal Assessment", "Temporal Confidence", "Liklihood", "Liklihood Confidence", "Subject", "Subject Confidence"]
dfentities = pd.DataFrame(columns = column_names)
new_row = {}

for entity in data['entityMentions']:
    new_row['Entity type'] = entity['type']
    new_row['Entity confidence'] = entity['confidence']
    new_row['Content'] = entity['text']['content']
    new_row['Entity#'] = entity['mentionId']

    if 'temporalAssessment' in entity:
        new_row['Temporal Assessment'] = entity['temporalAssessment']['value']
        new_row['Temporal Confidence'] = entity['temporalAssessment']['confidence']
        
    if 'certaintyAssessment' in entity:
        new_row['Liklihood'] = entity['certaintyAssessment']['value']
        new_row['Liklihood Confidence'] = entity['certaintyAssessment']['confidence']
  
    if 'subject' in entity:
        new_row['Subject'] = entity['subject']['value']
        new_row['Subject Confidence'] = entity['subject']['confidence']
  
    dfentities = dfentities.append(new_row, ignore_index=True) 
    
dfentities.to_csv('entities.csv')

## Let's summarize all the relationships of the "MEDICINE" entity.
I use the mentionId = 1 to parse the nlp outpuT and find relationships

In [21]:
def getEntity(id):
    for entity in data['entityMentions']:
        if entity['mentionId'] == id:
            return entity


In [24]:
import pandas as pd


column_names = ["Subject Type", "Subject Name", "Object Type", "Object Name", "Confidence"]
dfrels = pd.DataFrame(columns = column_names)
new_row = {}
print(len(data['relationships']))
for r in data['relationships']:
    #Get Subject
    subject = getEntity(r['subjectId'])
    obj = getEntity(r['objectId'])
    new_row["Subject Type"] = subject['type']
    new_row["Subject Name"] = subject['text']['content']
    new_row["Object Type"] = obj['type']
    new_row["Object Name"] = obj['text']['content']
    new_row["Confidence"] = r['confidence']
    dfrels = dfrels.append(new_row, ignore_index=True) 
    #print(new_row)
        
dfrels.to_csv('rel.csv')

62
